In [29]:
from selenium import webdriver
#import google_colab_selenium as gs
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from time import sleep
from selenium.common.exceptions import NoSuchElementException 

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [30]:
# # Instantiate options
options = Options()

# Add extra options
options.add_argument("--disable-infobars")  # Disable the infobars
options.add_argument("--disable-popup-blocking")  # Disable pop-ups
options.add_argument("--ignore-certificate-errors")  # Ignore certificate errors
options.add_argument("--incognito")  # Use Chrome in incognito mode


#driver = gs.Chrome(options=options)

In [31]:
driver=webdriver.Chrome(options=options)

In [32]:
driver.get('https://www.pinterest.com/')
sleep(3)

In [33]:
explore=driver.find_element(By.XPATH,'//div[@class="Eqh fev zI7 iyn Hsu"][2]/a')

In [34]:
explore.click()
sleep(3)

In [35]:
details={}
feed=[]

In [36]:
import queue

In [37]:
q=queue.Queue()

In [38]:
def getCapAndImage(a,q):
    driver=webdriver.Chrome(options=options)
    driver.get(a)
    text=driver.find_element(By.XPATH, "//h1").text
    image=driver.find_element(By.XPATH, "//img").get_attribute("src") 
    desc=driver.find_element(By.XPATH, "//img").get_attribute("alt")
    user=driver.find_element(By.XPATH, '//a[@data-test-id="creator-profile-link"]/div/div/div').text
    q.put([image,text,desc,user])
    details[image]=[user,text,desc]
    driver.quit()
    

In [39]:
answer=input("Do you want anything in the search or do you want the explore page? (Yes or No) ")
if(answer=="Yes"):
    x=input("What should I search? ")
    bar=driver.find_element(By.XPATH, "//input")
    bar.send_keys(x)
    bar.send_keys(Keys.RETURN)

In [40]:
current_scroll_position =0

In [41]:
try:
    while True:
        # Scroll down the page
        driver.execute_script(f"window.scrollTo(0, {current_scroll_position + 1500});")
        current_scroll_position += 1500
        
        # Wait for a few seconds to allow new content to load
        sleep(5)
        
        # Find elements
        elements = driver.find_elements(By.CSS_SELECTOR, ".Yl-.MIw.Hb7")
        if not elements:
            try:
                driver.find_elements(By.XPATH, '//div[@role="listitem"]')
            except:
                print("No more elements found.")
            break

        new_links_found = False
        for element in elements:
            try:
                a_tag = element.find_element(By.TAG_NAME, "a")
                href = a_tag.get_attribute("href")
                if href not in feed:
                    feed.append(href)
                    new_links_found = True
            except Exception as e:
                print(f"Error extracting link: {e}")

        # Check if no new content was loaded
        if not new_links_found:
            print("No new content loaded, breaking out.")
            break

finally:
    driver.quit()

print("Collected links:", feed)

No new content loaded, breaking out.
Collected links: ['https://www.pinterest.com/pin/300052393937727225/', 'https://www.pinterest.com/pin/939070959795692652/', 'https://www.pinterest.com/pin/537828380509349430/', 'https://www.pinterest.com/pin/15833036183466577/', 'https://www.pinterest.com/pin/6473993209565780/', 'https://www.pinterest.com/pin/756464068692957689/', 'https://www.pinterest.com/pin/662169951475140699/', 'https://www.pinterest.com/pin/978336719046763914/']


In [42]:
driver.quit()

In [43]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor() as exe:
        exe.map(lambda a: getCapAndImage(a, q), feed)

In [44]:
import pandas as pd
import numpy as np

In [45]:
ls=[]

In [46]:
while not q.empty():
    x=q.get()
    ls.append(x)

In [47]:
all=np.array(ls)

In [48]:
columns = ['Image', 'Caption', 'Description','User']

pinterest=pd.DataFrame(all,columns=columns)

ValueError: Shape of passed values is (0, 1), indices imply (0, 4)

In [25]:
pinterest

,Image,Caption,Description,User
0,https://i.pinimg.com/30x30_RS/78/20/5c/78205cb...,Homemade Cinnamon & Churros Ice Cream: Spice a...,FrostedFusions avatar link,FrostedFusions
1,https://i.pinimg.com/30x30_RS/c6/00/7b/c6007bc...,Homemade Choco Tacos,Crowded Kitchen avatar link,Crowded Kitchen
2,https://i.pinimg.com/236x/32/dc/f5/32dcf59affb...,Chocolate Orange Ice Cream,chocolate orange ice cream in a can with text ...,From The Larder | Gluten-Free Recipes


In [26]:
file=input("name of the file to save into")

In [27]:
pinterest.to_csv(file+".csv", index=False)